In [2]:
import torch
import importlib.util
import sys
import gc

# Add the BEATs module path
sys.path.append("/scratch/rd3629/ml-project/AudioSet-classification/beats")

# Define the path to the BEATs.py file
BEATs_path = "/scratch/rd3629/ml-project/AudioSet-classification/beats/BEATs.py"

# Load the BEATs module
spec = importlib.util.spec_from_file_location("BEATs", BEATs_path)
BEATs_module = importlib.util.module_from_spec(spec)
sys.modules["BEATs"] = BEATs_module
spec.loader.exec_module(BEATs_module)

# Import classes from the module
BEATs = BEATs_module.BEATs
BEATsConfig = BEATs_module.BEATsConfig

print("BEATs and BEATsConfig imported successfully!")


BEATs and BEATsConfig imported successfully!


In [3]:

# Load the checkpoint and initialize the model
checkpoint = torch.load('/scratch/rd3629/ml-project/AudioSet-classification/Pre-trained-model/BEATs_iter3_plus_AS2M.pt')


/state/partition1/job-54139371/ipykernel_2728384/803150140.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('/scratch/rd3629/ml-project/AudioSet-c

In [4]:
cfg = BEATsConfig(checkpoint['cfg'])
BEATs_model = BEATs(cfg)
BEATs_model.load_state_dict(checkpoint['model'])
BEATs_model.eval()

/ext3/miniconda3/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


BEATs(
  (post_extract_proj): Linear(in_features=512, out_features=768, bias=True)
  (patch_embedding): Conv2d(1, 512, kernel_size=(16, 16), stride=(16, 16), bias=False)
  (dropout_input): Dropout(p=0.1, inplace=False)
  (encoder): TransformerEncoder(
    (pos_conv): Sequential(
      (0): Conv1d(768, 768, kernel_size=(128,), stride=(1,), padding=(64,), groups=16)
      (1): SamePad()
      (2): GELU(approximate='none')
    )
    (layers): ModuleList(
      (0): TransformerSentenceEncoderLayer(
        (self_attn): MultiheadAttention(
          (dropout_module): Dropout(p=0.1, inplace=False)
          (relative_attention_bias): Embedding(320, 12)
          (k_proj): Linear(in_features=768, out_features=768, bias=True)
          (v_proj): Linear(in_features=768, out_features=768, bias=True)
          (q_proj): Linear(in_features=768, out_features=768, bias=True)
          (out_proj): Linear(in_features=768, out_features=768, bias=True)
          (grep_linear): Linear(in_features=64, out

In [5]:

# Ensure the model is on GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BEATs_model.to(device)

# Load audio input and ensure it is on the correct device
audio_input = torch.load('/scratch/rd3629/ml-project/AudioSet-classification/Data/train/resampled_waveforms.pt')
audio_input = audio_input.squeeze().to(device)
print(audio_input.shape)

/state/partition1/job-54139371/ipykernel_2728384/2257568278.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  audio_input = torch.load('/scratch/rd3629/ml-project/AudioSet

torch.Size([20550, 160172])


In [6]:

# Define the batch size
batch_size = 64  # Reduce batch size to prevent memory issues

# Get the number of samples
num_samples = audio_input.size(0)
representations = []

# Disable gradient calculations
with torch.no_grad():
    for i in range(0, num_samples, batch_size):
        batch = audio_input[i:i + batch_size]  # Slice the tensor for the current batch
        print(f"Processing batch {i // batch_size + 1}, shape: {batch.shape}")

        # Extract features
        representation = BEATs_model.extract_features(batch)[-1].detach().cpu()
        representations.append(representation)

        # Free memory
        del batch, representation
        torch.cuda.empty_cache()
        gc.collect()

# Concatenate all representations
total_representation = torch.cat(representations, dim=0)

print("Total representation shape:", total_representation.shape)



Processing batch 1, shape: torch.Size([64, 160172])


64it [00:02, 22.76it/s]


Processing batch 2, shape: torch.Size([64, 160172])


64it [00:00, 765.26it/s]


Processing batch 3, shape: torch.Size([64, 160172])


...

Processing batch 321, shape: torch.Size([64, 160172])


64it [00:00, 719.16it/s]


Processing batch 322, shape: torch.Size([6, 160172])


6it [00:00, 600.99it/s]


Total representation shape: torch.Size([20550, 496, 768])


In [7]:
# Save representation to file
torch.save(total_representation, 'train_representation.pt')